<br/>
<center>
<img src="https://www.etu.edu.tr/files/logolar/standart_logo/yatay/en/tobb_etu_yatay_en.png"
width="500" height="150"/>
</center>
<br/>



# BIL401/501 - Colab 1

Cheating will result in zero grade, whereas disciplinary actions may also be taken. Late submissions are not allowed. Our purpose is to introduce you to **SPARK**. Try to enjoy it.

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 51.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=ff22eee0ba13732a1c3a9d57fbe5139a1ece4ed4b33a477ffcd7a8d6dee1d36f
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sql = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
data = ['A B C', 'B C D', 'C D', 'D A']
links = spark.sparkContext.parallelize(data)

In [ ]:
# Key/value pairs
links = links.map(lambda x: (x.split(' ')[0], x.split(' ')[1:]))
print(links.collect())
 
# Find node count
N = links.count()
print(N)

# Create and initialize the ranks
ranks = links.map(lambda node: (node[0],1.0/N))
print(ranks.collect())

[('A', ['B', 'C']), ('B', ['C', 'D']), ('C', ['D']), ('D', ['A'])]
4
[('A', 0.25), ('B', 0.25), ('C', 0.25), ('D', 0.25)]


In [ ]:
ITERATIONS=20
for i in range(ITERATIONS):
    # Join graph info with rank info and propogate to all neighbors rank scores (rank/(number of neighbors)
    # And add up ranks from all in-coming edges
    ranks = links.join(ranks).flatMap(lambda x : [(i, float(x[1][1])*0.8/len(x[1][0])) for i in x[1][0]])\qq
    .reduceByKey(lambda x,y: x+y)
    print(ranks.sortByKey().collect())



[('A', 0.25), ('B', 0.125), ('C', 0.25), ('D', 0.375)]
[('A', 0.375), ('B', 0.125), ('C', 0.1875), ('D', 0.3125)]
[('A', 0.3125), ('B', 0.1875), ('C', 0.25), ('D', 0.25)]
[('A', 0.25), ('B', 0.15625), ('C', 0.25), ('D', 0.34375)]
[('A', 0.34375), ('B', 0.125), ('C', 0.203125), ('D', 0.328125)]
[('A', 0.328125), ('B', 0.171875), ('C', 0.234375), ('D', 0.265625)]
[('A', 0.265625), ('B', 0.1640625), ('C', 0.25), ('D', 0.3203125)]
[('A', 0.3203125), ('B', 0.1328125), ('C', 0.21484375), ('D', 0.33203125)]
[('A', 0.33203125), ('B', 0.16015625), ('C', 0.2265625), ('D', 0.28125)]
[('A', 0.28125), ('B', 0.166015625), ('C', 0.24609375), ('D', 0.306640625)]
[('A', 0.306640625), ('B', 0.140625), ('C', 0.2236328125), ('D', 0.3291015625)]
[('A', 0.3291015625), ('B', 0.1533203125), ('C', 0.2236328125), ('D', 0.2939453125)]
[('A', 0.2939453125), ('B', 0.16455078125), ('C', 0.2412109375), ('D', 0.30029296875)]
[('A', 0.30029296875), ('B', 0.14697265625), ('C', 0.229248046875), ('D', 0.323486328125)]
[(

In [ ]:
df1 = sql.createDataFrame([('20BJR',1,2), ('20BJR',2,8),('20BJR',3,11),\
                           ('34AB',3,20),('34AB',4,23),('34AB',5,28),\
                          ('08CC',1,100),('08CC',3,104),('08CC',4,115)],\
                           ['plaka','kamera','süre'])


In [ ]:
df1.show()

+-----+------+----+
|plaka|kamera|süre|
+-----+------+----+
|20BJR|     1|   2|
|20BJR|     2|   8|
|20BJR|     3|  11|
| 34AB|     3|  20|
| 34AB|     4|  23|
| 34AB|     5|  28|
| 08CC|     1| 100|
| 08CC|     3| 104|
| 08CC|     4| 115|
+-----+------+----+



In [ ]:

df2 = sql.createDataFrame([(1,2, 5 ), (2,3, 5 ), (1,3,10 ), (3,4, 10 ), (4,5,10 )],[ 'kamera1', 'kamera2','limit'])

In [ ]:
df2.show()

+-------+-------+-----+
|kamera1|kamera2|limit|
+-------+-------+-----+
|      1|      2|    5|
|      2|      3|    5|
|      1|      3|   10|
|      3|      4|   10|
|      4|      5|   10|
+-------+-------+-----+



In [ ]:
joined = df1.alias("a").join(df1.alias("b"), \
    [col("a.plaka") == col("b.plaka"),  col("a.kamera") != col("b.kamera")] ,"inner") \
    .select(col("a.plaka"),col("a.kamera").alias("kameraA"), \
      col("a.süre").alias("süreA"),  col("b.kamera").alias("kameraB"), \
      col("b.süre").alias("süreB")) 

In [ ]:
joined.show()

+-----+-------+-----+-------+-----+
|plaka|kameraA|süreA|kameraB|süreB|
+-----+-------+-----+-------+-----+
| 08CC|      1|  100|      3|  104|
| 08CC|      1|  100|      4|  115|
| 08CC|      3|  104|      1|  100|
| 08CC|      3|  104|      4|  115|
| 08CC|      4|  115|      1|  100|
| 08CC|      4|  115|      3|  104|
|20BJR|      1|    2|      2|    8|
|20BJR|      1|    2|      3|   11|
|20BJR|      2|    8|      1|    2|
|20BJR|      2|    8|      3|   11|
|20BJR|      3|   11|      1|    2|
|20BJR|      3|   11|      2|    8|
| 34AB|      3|   20|      4|   23|
| 34AB|      3|   20|      5|   28|
| 34AB|      4|   23|      3|   20|
| 34AB|      4|   23|      5|   28|
| 34AB|      5|   28|      3|   20|
| 34AB|      5|   28|      4|   23|
+-----+-------+-----+-------+-----+



In [ ]:
last = joined.join(df2 , (joined.kameraA == df2.kamera1 ) & (joined.kameraB == df2.kamera2 )  , 'right' )

In [ ]:
last.show()

+-----+-------+-----+-------+-----+-------+-------+-----+
|plaka|kameraA|süreA|kameraB|süreB|kamera1|kamera2|limit|
+-----+-------+-----+-------+-----+-------+-------+-----+
|20BJR|      2|    8|      3|   11|      2|      3|    5|
|20BJR|      1|    2|      2|    8|      1|      2|    5|
|20BJR|      1|    2|      3|   11|      1|      3|   10|
| 08CC|      1|  100|      3|  104|      1|      3|   10|
| 34AB|      3|   20|      4|   23|      3|      4|   10|
| 08CC|      3|  104|      4|  115|      3|      4|   10|
| 34AB|      4|   23|      5|   28|      4|      5|   10|
+-----+-------+-----+-------+-----+-------+-------+-----+



In [ ]:
last.filter(last['süreB'] - last['süreA'] > last['limit'] ).select('plaka').show()

+-----+
|plaka|
+-----+
|20BJR|
| 08CC|
+-----+



In [ ]:
last.printSchema()

root
 |-- plaka: string (nullable = true)
 |-- kamera: long (nullable = true)
 |-- süre: long (nullable = true)
 |-- kamera2: long (nullable = true)
 |-- süre2: long (nullable = true)
 |-- kamera1: long (nullable = true)
 |-- kamera2: long (nullable = true)
 |-- limit: long (nullable = true)



#### Read File ( 10 points )

In [ ]:
# Fill this part
txt =  sc.textFile("pg100.txt")

In [ ]:
txt

#### FlatMap ( 20 points )

In [ ]:
# Fill this part
flatmapped_datacontext = txt.flatMap(lambda line: line.split(" ")) 

In [ ]:
flatmapped_datacontext

#### Map ( 20 points )

In [ ]:
# Fill this part
mapped_datacontext = flatmapped_datacontext.map(lambda word: (word, 1))

In [ ]:
mapped_datacontext

#### Reduce ( 20 points )

In [ ]:
# Fill this part
counts = mapped_datacontext.reduceByKey(lambda a, b: a + b)

In [ ]:
counts.take(10)

#### Take() first 10 key-values sorted by descending. (10 points)

In [ ]:
# Fill this part
first_10 = counts.map(lambda a : (a[1],a[0])).sortByKey(False).take(10)

#### Number of Unique Words ( 10 points ) 

In [ ]:
# Fill this part
counts.count()

### Upload Result


Please follow the intructions below before submit your colab. 



Once you obtained the desired results, you are going to submit your colab via **UZAK** as a **.ipynb** file. Name your document as Colab1_studentid.ipynb (ex: "HW1_1234567.ipynb").

.....